# Final Project
#### By Shubhankar Kumar

### 1. Loading the dataset, cleaning and basic EDA

In [1]:
def load_and_display_data(folder_path):
    df = spark.read.parquet(folder_path)
    df.printSchema()
    df.show(5)
    return df

In [2]:
commits_df = load_and_display_data("gs://msca-bdp-data-open/final_project_git/commits")

root
 |-- commit: string (nullable = true)
 |-- tree: string (nullable = true)
 |-- parent: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- author: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- time_sec: long (nullable = true)
 |    |-- tz_offset: long (nullable = true)
 |    |-- date: struct (nullable = true)
 |    |    |-- seconds: long (nullable = true)
 |    |    |-- nanos: long (nullable = true)
 |-- committer: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- time_sec: long (nullable = true)
 |    |-- tz_offset: long (nullable = true)
 |    |-- date: struct (nullable = true)
 |    |    |-- seconds: long (nullable = true)
 |    |    |-- nanos: long (nullable = true)
 |-- subject: string (nullable = true)
 |-- message: string (nullable = true)
 |-- trailer: array (nullable = true)
 |    |-- element: struct (contains

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------+
|              commit|                tree|              parent|              author|           committer|             subject|             message|             trailer|difference|difference_truncated|           repo_name|encoding|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------+
|aa358905a1b12c6fa...|df3f8bf61bf1cb0df...|[ea230a45a0e97e4d...|{conda-forge-coor...|{conda-forge-coor...|Updated the qceng...|Updated the qceng...|                  []|        []|                true|[conda-forge/feed...|    null|
|5a6b6d6d29489f858...|ff89accb7e283ca88...|[4ee369feb64ee97d...|{Rob All

In [3]:
# Keeping only the relevant columns
from pyspark.sql.functions import col
selected_columns = ["author", "committer", "subject", "message", "repo_name", "commit",'difference']
commits_df_selected = commits_df.select(*selected_columns)

commits_df_selected.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|              author|           committer|             subject|             message|           repo_name|              commit|difference|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|{conda-forge-coor...|{conda-forge-coor...|Updated the qceng...|Updated the qceng...|[conda-forge/feed...|aa358905a1b12c6fa...|        []|
|{Rob Allen, 7e09c...|{Rob Allen, 7e09c...|Merge remote-trac...|Merge remote-trac...|[MadCat34/zend-es...|5a6b6d6d29489f858...|        []|
|{Zhihui Zhang, 96...|{Zhihui Zhang, 96...|provide hook to o...|provide hook to o...|[pscedu/slash2-st...|6b6ac3b8ab7363b22...|        []|
|{conda-forge-coor...|{conda-forge-coor...|Updated the mailc...|Updated the mailc...|[conda-forge/feed...|e26e1f63938b983ce...|        []|
|{armaneshaghi, f6...|{arma

In [4]:
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import col, regexp_extract
commits_df_selected = commits_df_selected.withColumn(
    "date",
    col("committer.time_sec")
)
commits_df_selected = commits_df_selected.withColumn("timestamp", from_unixtime("date"))
commits_df_selected = commits_df_selected.drop("date")

from pyspark.sql.functions import year
import matplotlib.pyplot as plt

commits_df_selected = commits_df_selected.withColumn("year", year(col("timestamp")))

commits_df_new = commits_df_selected.filter((year("timestamp") >= 2000) & (year("timestamp") <= 2023))
commits_df_new = commits_df_new.drop("year", "month")

In [5]:
del commits_df_selected

In [5]:
contents_df = load_and_display_data("gs://msca-bdp-data-open/final_project_git/contents")

root
 |-- id: string (nullable = true)
 |-- size: long (nullable = true)
 |-- content: string (nullable = true)
 |-- binary: boolean (nullable = true)
 |-- copies: long (nullable = true)



+--------------------+-----+--------------------+------+------+
|                  id| size|             content|binary|copies|
+--------------------+-----+--------------------+------+------+
|d5b1049fdaa182fa5...| 1570|{"version":3,"sou...| false|   256|
|896830f9ea31efd6b...|18616|                null|  true|     1|
|bf1e2a8490344601c...|15580|                null|  true|     1|
|e5976431eba91aa73...| 3328|                null|  true|     1|
|311532e41682cab22...| 8970|                null|  true|     1|
+--------------------+-----+--------------------+------+------+
only showing top 5 rows



In [6]:
selected_columns = ["id", "size", "content"]
contents_df_selected = contents_df.select(*selected_columns)
contents_df_selected = contents_df_selected.na.drop(subset=["content"])
contents_df_selected.show(5)

+--------------------+----+--------------------+
|                  id|size|             content|
+--------------------+----+--------------------+
|d5b1049fdaa182fa5...|1570|{"version":3,"sou...|
|c62c391e494770dbd...| 256|# -*- encoding: u...|
|a5f0a6fdebc99d68c...| 256|# Kconfig - ST ST...|
|919df5f2fe84b4a04...| 256|#include <stdio.h...|
|3704f81ed29edd51a...| 256|% Generated by ro...|
+--------------------+----+--------------------+
only showing top 5 rows



In [7]:
del contents_df

In [8]:
files_df = load_and_display_data("gs://msca-bdp-data-open/final_project_git/files")

root
 |-- repo_name: string (nullable = true)
 |-- ref: string (nullable = true)
 |-- path: string (nullable = true)
 |-- mode: long (nullable = true)
 |-- id: string (nullable = true)
 |-- symlink_target: string (nullable = true)



+--------------------+-----------------+--------------------+-----+--------------------+--------------+
|           repo_name|              ref|                path| mode|                  id|symlink_target|
+--------------------+-----------------+--------------------+-----+--------------------+--------------+
|    enzbang/diouzhtu|refs/heads/master|gwiad_wiki_servic...|33261|49365044eed287691...|          null|
|TheMrNomis/Latex-...|refs/heads/master|             LFM.php|33261|ef8cb78feed7f2111...|          null|
|TheMrNomis/Latex-...|refs/heads/master|PHP/LatexFlavored...|33261|d989ce59652f57efa...|          null|
|    xurigan/uexJPush|refs/heads/master|EUExJPush/EUExJPu...|33261|85268b90caa19efa2...|          null|
|    xurigan/uexJPush|refs/heads/master|EUExJPush/uexJPus...|33261|e1623bb9d8dc7db60...|          null|
+--------------------+-----------------+--------------------+-----+--------------------+--------------+
only showing top 5 rows



In [9]:
del files_df

In [10]:
languages_df = load_and_display_data("gs://msca-bdp-data-open/final_project_git/languages")

root
 |-- repo_name: string (nullable = true)
 |-- language: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- bytes: long (nullable = true)



+-------------------+------------+
|          repo_name|    language|
+-------------------+------------+
|  lemi136/puntovent|   [{C, 80}]|
|     taxigps/nctool| [{C, 4461}]|
|        ahy1/strbuf| [{C, 5573}]|
|nleiten/mod_rpaf-ng|[{C, 30330}]|
|kmcallister/alameda|[{C, 17077}]|
+-------------------+------------+
only showing top 5 rows



In [11]:
languages_df = languages_df.na.drop(subset=["language"])

In [12]:
licenses_df = load_and_display_data("gs://msca-bdp-data-open/final_project_git/licenses")

root
 |-- repo_name: string (nullable = true)
 |-- license: string (nullable = true)

+--------------------+------------+
|           repo_name|     license|
+--------------------+------------+
|autarch/Dist-Zill...|artistic-2.0|
|thundergnat/Prime...|artistic-2.0|
|kusha-b-k/Turabia...|artistic-2.0|
|onlinepremiumoutl...|artistic-2.0|
|huangyuanlove/Lia...|artistic-2.0|
+--------------------+------------+
only showing top 5 rows



In [13]:
licenses_df = licenses_df.na.drop(subset=["license"])

In [ ]:
commits_df_new.write.parquet("gs://msca-bdp-students-bucket/shared_data/shubhankark/commits", mode="overwrite")

In [2]:
#contents_df_selected.write.parquet("gs://msca-bdp-students-bucket/shared_data/shubhankark/contents", mode="overwrite")

In [16]:
languages_df.write.parquet("gs://msca-bdp-students-bucket/shared_data/shubhankark/languages", mode="overwrite")

In [17]:
licenses_df.write.parquet("gs://msca-bdp-students-bucket/shared_data/shubhankark/licenses", mode="overwrite")